In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cascade-cup-debye/Cascade_cup/sample_submission.csv
/kaggle/input/cascade-cup-debye/Cascade_cup/test_age_dataset.csv
/kaggle/input/cascade-cup-debye/Cascade_cup/train_age_dataset.csv


In [2]:
import pandas as pd
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
from sklearn.utils import class_weight

In [3]:
train = pd.read_csv('/kaggle/input/Cascade_cup/train_age_dataset.csv')
test = pd.read_csv('/kaggle/input/Cascade_cup/test_age_dataset.csv')
sample = pd.read_csv('/kaggle/input/Cascade_cup/sample_submission.csv')

In [4]:
train = pd.get_dummies(train, columns=['tier','gender'])
test = pd.get_dummies(test, columns=['tier','gender'])

# REMEMBER TO ADD 1 IN THE END

train['age_group'] = train['age_group']-1

In [5]:
feature_cols = [col for col in train.columns.tolist() if col not in ['age_group','num_of_hashtags_per_action','emoji_count_per_action','tier_1','tier_3']]
target_cols = ['age_group']

In [6]:
# KFOLD
skf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

for i, (trn, val) in enumerate(skf.split(train[target_cols], train[target_cols])):
    train.loc[val, 'kfold'] = i
train['kfold'] = train['kfold'].astype(int)

In [7]:
def run_training():
    oof = np.zeros((train.shape[0], 4))
    pred = np.zeros((test.shape[0], 4))
    
    for fold in range(5):
        
        print(f"\nStarting FOLD: {fold}")
        start = time.time()
        

        trn_idx = train['kfold'] != fold
        val_idx = train['kfold'] == fold
        trn = train.loc[trn_idx, :]
        val = train.loc[val_idx, :]

        xtr, ytr = trn[feature_cols].values, trn[target_cols].values
        xval, yval = val[feature_cols].values, val[target_cols].values
        xtest = test[feature_cols].values

        model = RandomForestClassifier(n_estimators=400, max_depth=15, max_features='sqrt')
        print("Fitting model ...")
        model.fit(xtr, ytr.reshape(-1,))
        
        print("Calculating training preds ...")
        train_preds = model.predict(xtr)
        print("Calculating validation preds ...")
        val_preds = model.predict(xval)
        print("Training Accuracy Score - ", accuracy_score(ytr, train_preds ))
        print("Training F1 Score - ", f1_score(ytr, train_preds, average='weighted'))
        
        print("Validation Accuracy Score - ", accuracy_score(yval, val_preds))
        print("Validation F1 Score - ", f1_score(yval, val_preds, average='weighted'))
        
        oof[val_idx, :] += model.predict_proba(xval)
        
        # Test preds
        pred += model.predict_proba(xtest)/5
        
        print(f"FOLD {fold} completed in {time.time()-start} seconds")
        
    return oof, pred

In [8]:
oof_xgb, pred_xgb = run_training()


Starting FOLD: 0
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8536669555945906
Training F1 Score -  0.853334566186067
Validation Accuracy Score -  0.7676730486008837
Validation F1 Score -  0.7668855227646146
FOLD 0 completed in 558.9609444141388 seconds

Starting FOLD: 1
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8550732419502891
Training F1 Score -  0.854285865037793
Validation Accuracy Score -  0.7690742104401899
Validation F1 Score -  0.7675117328360401
FOLD 1 completed in 553.8857092857361 seconds

Starting FOLD: 2
Fitting model ...
Calculating training preds ...
Calculating validation preds ...
Training Accuracy Score -  0.8539076762583674
Training F1 Score -  0.8532621969979965
Validation Accuracy Score -  0.7684479672717975
Validation F1 Score -  0.7666508490628322
FOLD 2 completed in 550.2530088424683 seconds

Starting FOLD: 3
Fitting model ...


In [9]:
def find_oof_score(oof):
    predictions = oof.argmax(axis=1)
    print("OOF Accuracy Score - ", accuracy_score(train[target_cols], predictions))
    print("OOF F1 Score - ", f1_score(train[target_cols], predictions, average='weighted'))

In [10]:
find_oof_score(oof_xgb)

OOF Accuracy Score -  0.7679559480196451
OOF F1 Score -  0.7668208191337091


In [11]:
final_preds = pred_xgb.argmax(axis=1)+1
pred_csv = pd.DataFrame(final_preds.reshape(-1), columns=['prediction'] )
pred_csv

,prediction
0,1
1,1
2,1
3,2
4,1
...,...
54315,1
54316,1
54317,4
54318,1


In [12]:
pred_csv.value_counts()

prediction
1             33637
2              9564
3              7503
4              3616
dtype: int64

In [13]:
pred_csv.to_csv('sub_rnd2.csv', index=False)

In [14]:
np.save('oof_rnd.npy', oof_xgb)
np.save('pred_rnd.npy', pred_xgb)